In [2]:
import torch
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import rosbag
import math
from scipy.spatial.transform import Rotation as R


PermissionError: [Errno 13] Permission denied: '/usr/local/lib/python3.8/dist-packages/pyzed-3.7.dist-info'

In [ ]:
def load_frame(fname):
    frame = cv2.imread(fname)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    frame = cv2.resize(frame, (1920, 1080))
    return frame

In [ ]:
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_1000)
arucoParams = cv2.aruco.DetectorParameters_create()

def detect(frame):
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)
    corners = np.array(corners).reshape((4, 2)).astype(int)
    topLeft, topRight, bottomRight, bottomLeft = corners
    
    cX = int((topLeft[0] + bottomRight[0]) / 2.0)
    cY = int((topLeft[1] + bottomRight[1]) / 2.0)
    center = np.array([cX, cY])
    return center

In [ ]:
img_h, img_w = 1080, 1920
fov_h, fov_v = 84, 53  # degrees

In [ ]:
fnames = ['samples/frame_1800.jpg', 'samples/frame_2800.jpg', 'samples/frame_4440.jpg']
frames = [load_frame(fname) for fname in fnames]
centers = np.array([detect(frame) for frame in frames])
frames_dot = [cv2.circle(frame.copy(), tuple(center), 10, (255, 0, 0), -1) for frame, center in zip(*[frames, centers])]

In [ ]:
grid = np.vstack([
    np.hstack(frames),
    np.hstack(frames_dot),
])
plt.figure(figsize=(20, 20)) 
plt.imshow(grid)

In [ ]:
def parse_log(fname):
    with open(fname) as f:
        lines = f.readlines()
    data = [eval(l.strip()) for l in lines]
    df = pd.DataFrame(data)
    pos = [df['lt'].mean(), df['ln'].mean(), df['al'].mean()]
    cov = [df['lt'].var(), df['ln'].var(), df['al'].var()]
    pos = np.array(pos)
    cov = np.array(cov) * np.eye(3)
    return pos, cov

In [ ]:
n_pos, n_cov = parse_log('Node Positioning GPS Data/R00-node1-left.log')

gps_points = [n_pos]
for i in [1,2,3]:
    o_pos, o_cov = parse_log('Node Positioning GPS Data/R00-node1-pos%d.log' % i)
    gps_points.append(o_pos)
gps_points = np.array(gps_points)

In [ ]:
with open('camera_info.json') as f:
    camera_info = json.load(f)
camera_info

In [ ]:
rot_df = pd.read_csv('20240108-224709_camera_rotation.csv', header=None)
rot = R.from_euler('zyx', rot_df.mean(), degrees=True)
rot.as_matrix()

In [ ]:
def gps2cartesian(gps_coordinates):
    """
    Convert GPS coordinates (latitude, longitude, altitude) to Cartesian coordinates (x, y, z),
    with all measurements in meters.
    
    Parameters:
    gps_coordinates (numpy array): An N x 3 array where each row contains latitude, longitude, and altitude.
                                   Latitude and Longitude should be in degrees, altitude in meters.
    
    Returns:
    numpy array: An N x 3 array of Cartesian coordinates (x, y, z) in mm.
    """
    # Earth's mean radius in meters
    R = 6371000

    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(gps_coordinates[:, 0])
    lon_rad = np.radians(gps_coordinates[:, 1])

    # Adjusted radius with altitude
    adjusted_radius = R + gps_coordinates[:, 2]

    # Calculate Cartesian coordinates
    x = adjusted_radius * np.cos(lat_rad) * np.cos(lon_rad)
    y = adjusted_radius * np.cos(lat_rad) * np.sin(lon_rad)
    z = adjusted_radius * np.sin(lat_rad)

    return np.column_stack((x, y, z)) * 1000

def gps2cartesian_torch(gps_coordinates, rad=6371000):
    lat_rad = torch.radians(gps_coordinates[:, 0])
    lon_rad = torch.radians(gps_coordinates[:, 1])

    # Adjusted radius with altitude
    adjusted_radius = rad + gps_coordinates[:, 2]

    # Calculate Cartesian coordinates
    x = adjusted_radius * torch.cos(lat_rad) * torch.cos(lon_rad)
    y = adjusted_radius * torch.cos(lat_rad) * torch.sin(lon_rad)
    z = adjusted_radius * torch.sin(lat_rad)

    return torch.stack((x, y, z), dim=1) * 1000

In [ ]:
torch.radians

In [ ]:
def cartesian2spherical(cart_coords):
    x, y, z = cart_coords
    r = np.sqrt(x**2 + y**2 + z**2)  # Radius
    theta = np.arctan2(y, x)  # Azimuthal angle
    phi = np.arccos(z / r)  # Polar angle

    # Ensure theta is between 0 and 2π
    #theta = theta % (2 * np.pi)

    theta = np.degrees(theta)
    phi = np.degrees(phi)
    if phi > 90:
        phi = 180 - phi
    return np.array([r, theta, phi])

In [ ]:
centers

In [ ]:
cart_points = gps2cartesian(gps_points)
cart_points

In [ ]:
local_points = (cart_points - cart_points[0,:]) @ rot.as_matrix()
local_points

In [ ]:
ox = -30
oy = -200
oz = 150

In [ ]:
def local2pixel(local_points, camera_info):
    X = -(local_points[:, 1] + oy)
    Y = -(local_points[:, 2] + oz)
    Z = (local_points[:, 0]) + ox
    u = (X/Z) * camera_info['left']['fx'] + camera_info['left']['cx']
    v = (Y/Z) * camera_info['left']['fy'] + camera_info['left']['cy']
    pixels = np.vstack([u, v]).T
    return pixels

In [ ]:
local2pixel(local_points, camera_info)

In [ ]:
u = (-O_{loc}^L[1] / O_{loc}^L[0]) * f_x + c_x 
v = (-O_{loc}^L[2] / O_{loc}^L[0]) * f_y + c_y

In [ ]:
local_points[1]

In [ ]:
X

In [ ]:
L = local_points[0] * 1000
X = -(L[1] + oy)
Y = -(L[2] + oz)
Z = (L[0]) + ox
(X/Z) * camera_info['left']['fx'] + camera_info['left']['cx']

In [ ]:
(Y/Z) * camera_info['left']['fy'] + camera_info['left']['cy']

In [ ]:
camera_info['left']['fx']

In [ ]:
#o_local = (o_pos - n_pos) @ R
o_local = local_points @ rot.as_matrix()
r, theta, phi = cartesian_to_spherical(o_local[3])
r, theta, phi

In [ ]:
# Point's spherical coordinates (in degrees)

# Normalize the angles relative to the FOV
# Assuming the center of the image corresponds to (theta_center, phi_center)
theta_center, phi_center = 0, 90  # degrees
normalized_theta = (theta - theta_center) / fov_h
normalized_phi = (phi - phi_center) / fov_v
print(normalized_theta, normalized_phi)

# Calculate the pixel coordinates
# Flip the Y-axis because pixel coordinates have the origin at the top-left corner
pixel_x = int((normalized_theta + 0.5) * img_w)
pixel_y = int((0.5 - normalized_phi) * img_h)

# Ensure the pixel coordinates are within the image bounds
#pixel_x = max(0, min(width - 1, pixel_x))
#pixel_y = max(0, min(height - 1, pixel_y))

# Output the pixel coordinates
print(f"Pixel Coordinates: (x, y) = ({pixel_x}, {pixel_y})")


In [ ]:
def pixel_to_spherical(x, y, W, H, fov_x, fov_y):
    # Normalize the pixel coordinates to -1 to 1
    nx = (x / (W - 1)) * 2 - 1  # Normalized to -1 to 1
    ny = (y / (H - 1)) * 2 - 1  # Normalized to -1 to 1

    # Calculate theta and phi based on the field of view
    theta = (nx * fov_x / 2) + theta_center  # Adjusted by the center of the FOV
    phi = (-ny * fov_y / 2) + phi_center    # Adjusted by the center of the FOV, flip the y-axis
    
    return theta, phi

# Example usage:
x, y = 498, 279 # Pixel coordinates
fov_x, fov_y = 84, 53  # Camera's field of view in degrees
theta_center, phi_center = 0, 90  # Center of the camera's field of view in degrees

theta, phi = pixel_to_spherical(x, y, img_w, img_h, fov_x, fov_y)
print("Theta (θ):", theta, "degrees")
print("Phi (φ):", phi, "degrees")
